In [2]:
from tokenizer import train_mopiece, MOPiece
from word_autoencoder import WordEncoder, WordDecoder
import torch as pt
from torch import nn
import pandas as pl
import regex as re
from tqdm import trange, tqdm, tqdm_notebook

device = ('cuda' if pt.cuda.is_available() else 'mps' if pt.backends.mps.is_available() else 'cpu')

In [3]:
suffixes = pl.read_csv('data/morphynet/suffixes.csv')['suffix']
prefixes = pl.read_csv('data/morphynet/prefixes.csv')['prefix']

train_mopiece('____tokenizer', ['data/text/bible.txt'], prefixes, suffixes, 3000, spm_model_type='bpe')

mopiece = MOPiece('____tokenizer')

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: 
  model_type: BPE
  vocab_size: 3000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 0
  eos_id: 1
  pad_id: 2
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0
  di

In [4]:
words = set()
reg = re.compile(r'([^\p{L}\p{M}\p{N}\s]+|\s)')
with open('data/text/bible.txt', mode='r', encoding='utf8') as file:
            for line in file.readlines():
                for word in reg.split(line):
                    if word == '' or word == ' ':
                        continue
                    word = word.lower()
                    words.add(word)

In [5]:
class WordDataset(pt.utils.data.Dataset):
    def __init__(self, words, mopiece, device=None):
        super().__init__()
        bos_id, eos_id = mopiece.bos_id(), mopiece.eos_id()
        self.words = []
        for word in words:
            prefix_ids, spm_ids, suffix_ids = mopiece.encode_word(word)
            self.words.append((
                pt.tensor([bos_id] + prefix_ids + [eos_id], dtype=pt.long, device=device), 
                pt.tensor([bos_id] + spm_ids    + [eos_id], dtype=pt.long, device=device), 
                pt.tensor([bos_id] + suffix_ids + [eos_id], dtype=pt.long, device=device)
            ))
        
    def __len__(self):
        return len(self.words)
    
    def __getitem__(self, index):
        return self.words[index]
    
class SequenceCrossEntropyLoss(nn.CrossEntropyLoss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs, reduction='sum')

    def forward(self, prefix_logits, spm_logits, suffix_logits, prefix_ids, spm_ids, suffix_ids):
        loss = 0.
        for prefix_logits_seq, spm_logits_seq, suffix_logits_seq, prefix_ids_seq, spm_ids_seq, suffix_ids_seq in zip(prefix_logits, spm_logits, suffix_logits, prefix_ids, spm_ids, suffix_ids):
            loss += (super().forward(prefix_logits_seq, prefix_ids_seq) + super().forward(spm_logits_seq, spm_ids_seq) + super().forward(suffix_logits_seq, suffix_ids_seq)) / (len(prefix_ids_seq) + len(spm_ids_seq) + len(suffix_ids_seq))
        return loss / prefix_logits.shape[0]
    
dset = WordDataset(words, mopiece, device=device)

def collate_fn(batch):
    return (pt.nn.utils.rnn.pad_sequence([prefix for prefix, spm, suffix in batch], batch_first=True, padding_value=mopiece.pad_id(), padding_side='right'),
            pt.nn.utils.rnn.pad_sequence([spm    for prefix, spm, suffix in batch], batch_first=True, padding_value=mopiece.pad_id(), padding_side='right'),
            pt.nn.utils.rnn.pad_sequence([suffix for prefix, spm, suffix in batch], batch_first=True, padding_value=mopiece.pad_id(), padding_side='right'))

In [6]:
encoder = WordEncoder(mopiece.vocab_size(), mopiece.pad_id(), 128, ffn_hidden_dim=256, expansion_factor=4).to(device)
decoder = WordDecoder(mopiece.vocab_size(), mopiece.pad_id(), mopiece.bos_id(), mopiece.eos_id(), 128, num_layers=6).to(device)

In [7]:
criterion = SequenceCrossEntropyLoss(ignore_index=mopiece.pad_id())

epochs = 5
batch_size = 128

loader = pt.utils.data.DataLoader(dset, batch_size, shuffle=True, drop_last=True, collate_fn=collate_fn)

optim = pt.optim.AdamW([
    {"params": encoder.parameters()},
    {"params": decoder.parameters()}
], weight_decay=0.01)
lr_scheduler = pt.optim.lr_scheduler.CosineAnnealingLR(optim, epochs)

encoder.train()
decoder.train()

log_perplexities = []
for epoch in (pbar := trange(epochs, desc='Epoch')):
    log_perplexity_sum = 0
    for prefix_ids, spm_ids, suffix_ids in tqdm(loader, desc='Training', leave=False):
        optim.zero_grad()

        embedding = encoder(prefix_ids, spm_ids, suffix_ids)

        prefix_logits, spm_logits, suffix_logits = decoder(prefix_ids[..., :-1], spm_ids[..., :-1], suffix_ids[..., :-1], embedding)

        loss = criterion(prefix_logits, spm_logits, suffix_logits, prefix_ids[..., 1:], spm_ids[..., 1:], suffix_ids[..., 1:])

        loss.backward()
        optim.step()

        log_perplexity_sum += loss.item()

    log_perplexity = log_perplexity_sum / len(loader)
    log_perplexities.append(log_perplexity)
    pbar.set_postfix_str(f'log-perplexity: {log_perplexity:.2f}')



Epoch:   0%|          | 0/5 [00:35<?, ?it/s]              


KeyboardInterrupt: 

In [28]:
from random import sample

with pt.no_grad():
    encoder.eval()
    decoder.eval()
    bos_id, eos_id = mopiece.bos_id(), mopiece.eos_id()

    word = sample(list(words), 1)[0]
    prefix_ids, spm_ids, suffix_ids = map(lambda x: [bos_id] + x + [eos_id], mopiece.encode_word(word))
    print('input:')
    print(prefix_ids, spm_ids, suffix_ids)
    print(word)


    embedding = encoder(pt.tensor(prefix_ids, dtype=pt.long, device=device), pt.tensor(spm_ids, dtype=pt.long, device=device), pt.tensor(suffix_ids, dtype=pt.long, device=device))

    out_prefix_ids, out_spm_ids, out_suffix_ids = decoder.inference(embedding)
    print('\noutput:')
    print(out_prefix_ids, out_spm_ids, out_suffix_ids)
    print(mopiece.decode_word(out_prefix_ids, out_spm_ids, out_suffix_ids))

input:
[0, 1] [0, 33, 1834, 2752, 1] [0, 1]
dophkah

output:
[0, 697, 1] [0, 1164, 149, 2544, 1] [0, 1]
supershutladah


In [14]:
import torch as pt
from torch import nn
import torch.nn.functional as F

from common import Encoder, Decoder, RoPEMHSA, MultiheadCrossAttention, FFN


In [15]:
class EnhancedWordEncoder(nn.Module):
    def __init__(self, vocab_size, pad_id, embedding_dim=256, spm_dim=512,
                 suffix_dim=256, prefix_dim=256, spm_layers=6, spm_heads=16,
                 spm_rope_cache=128, ffn_hidden_dim=1024, expansion_factor=4, dropout=0.2):
        super().__init__()

        self.vocab_size = vocab_size
        self.pad_id = pad_id
        self.dropout = nn.Dropout(dropout)

        self.spm_embedding = nn.Embedding(vocab_size[1], spm_dim, padding_idx=pad_id)
        nn.init.normal_(self.spm_embedding.weight, mean=0.0, std=0.02)

        self.spm_encoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'attn': RoPEMHSA(spm_dim, spm_dim // spm_heads, spm_dim // spm_heads,
                                spm_heads, dropout, spm_rope_cache),
                'ffn': FFN(spm_dim, ffn_hidden_dim, dropout),
                'norm1': nn.RMSNorm(spm_dim),
                'norm2': nn.RMSNorm(spm_dim)
            })
            for _ in range(spm_layers)
        ])

        self.spm_final_norm = nn.RMSNorm(spm_dim)

        self.spm_pooling_q = nn.Parameter(pt.randn(spm_heads, spm_dim // spm_heads) * 0.02)
        self.spm_pooling_k = nn.Linear(spm_dim, spm_dim)
        self.spm_pooling_v = nn.Linear(spm_dim, spm_dim)
        self.spm_pooling_proj = nn.Linear(spm_dim, embedding_dim)

        self.prefix_embedding = nn.Embedding(vocab_size[0], prefix_dim, padding_idx=pad_id)
        nn.init.normal_(self.prefix_embedding.weight, mean=0.0, std=0.02)

        self.prefix_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=prefix_dim,
                nhead=8,
                dim_feedforward=prefix_dim * 2,
                dropout=dropout,
                activation='gelu',
                batch_first=True,
                norm_first=True
            ),
            num_layers=3
        )

        self.prefix_pooling = nn.Sequential(
            nn.Linear(prefix_dim, embedding_dim),
            nn.LayerNorm(embedding_dim),
            nn.GELU()
        )

        self.suffix_embedding = nn.Embedding(vocab_size[2], suffix_dim, padding_idx=pad_id)
        nn.init.normal_(self.suffix_embedding.weight, mean=0.0, std=0.02)

        self.suffix_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=suffix_dim,
                nhead=8,
                dim_feedforward=suffix_dim * 2,
                dropout=dropout,
                activation='gelu',
                batch_first=True,
                norm_first=True
            ),
            num_layers=3
        )

        self.suffix_pooling = nn.Sequential(
            nn.Linear(suffix_dim, embedding_dim),
            nn.LayerNorm(embedding_dim),
            nn.GELU()
        )

        self.integration_net = nn.Sequential(
            nn.Linear(embedding_dim * 3, embedding_dim * 2),
            nn.LayerNorm(embedding_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim * 2, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )

    def forward(self, prefix_ids, spm_ids, suffix_ids):
        spm_pad_mask = spm_ids != self.pad_id
        spm_attn_mask = spm_pad_mask.unsqueeze(-1) & spm_pad_mask.unsqueeze(-2)

        spm_emb = self.dropout(self.spm_embedding(spm_ids))

        for layer in self.spm_encoder_layers:
            residual = spm_emb
            spm_emb = layer['norm1'](spm_emb)
            spm_emb = residual + layer['attn'](spm_emb, spm_attn_mask)

            # FFN with residual
            residual = spm_emb
            spm_emb = layer['norm2'](spm_emb)
            spm_emb = residual + layer['ffn'](spm_emb)

        spm_emb = self.spm_final_norm(spm_emb)

        batch_shape = spm_emb.shape[:-2]
        L = spm_emb.shape[-2]

        spm_k = self.spm_pooling_k(spm_emb).view(*batch_shape, L, *self.spm_pooling_q.shape).transpose(-2, -3)
        spm_v = self.spm_pooling_v(spm_emb).view(*batch_shape, L, *self.spm_pooling_q.shape).transpose(-2, -3)

        q_expanded = self.spm_pooling_q.view(*(1,)*len(batch_shape), self.spm_pooling_q.shape[0], 1, self.spm_pooling_q.shape[1])

        spm_word_emb = self.spm_pooling_proj(
            F.scaled_dot_product_attention(
                q_expanded,
                spm_k,
                spm_v,
                spm_pad_mask.view(*batch_shape, 1, 1, L)
            ).flatten(-3)
        )

        prefix_pad_mask = prefix_ids != self.pad_id
        prefix_mask = ~prefix_pad_mask

        prefix_emb = self.dropout(self.prefix_embedding(prefix_ids))
        prefix_emb = self.prefix_encoder(prefix_emb, src_key_padding_mask=prefix_mask)

        prefix_mask_expanded = prefix_pad_mask.unsqueeze(-1).float()
        prefix_sum = (prefix_emb * prefix_mask_expanded).sum(dim=-2)
        prefix_count = prefix_mask_expanded.sum(dim=-2).clamp(min=1.0)
        prefix_pooled = prefix_sum / prefix_count

        prefix_word_emb = self.prefix_pooling(prefix_pooled)

        suffix_pad_mask = suffix_ids != self.pad_id
        suffix_mask = ~suffix_pad_mask

        suffix_emb = self.dropout(self.suffix_embedding(suffix_ids))
        suffix_emb = self.suffix_encoder(suffix_emb, src_key_padding_mask=suffix_mask)

        suffix_mask_expanded = suffix_pad_mask.unsqueeze(-1).float()
        suffix_sum = (suffix_emb * suffix_mask_expanded).sum(dim=-2)
        suffix_count = suffix_mask_expanded.sum(dim=-2).clamp(min=1.0)
        suffix_pooled = suffix_sum / suffix_count

        suffix_word_emb = self.suffix_pooling(suffix_pooled)

        word_emb = self.integration_net(
            pt.cat([spm_word_emb, prefix_word_emb, suffix_word_emb], dim=-1)
        )

        return word_emb

In [16]:
class EnhancedWordDecoder(nn.Module):
    def __init__(self, vocab_size, pad_id, bos_id, eos_id, embedding_dim=256,
                 spm_dim=512, suffix_dim=256, prefix_dim=256, num_layers=6,
                 num_heads=8, ffn_dim=1024, dropout=0.2, rope_cache=128):
        super().__init__()

        self.vocab_size = vocab_size
        self.pad_id = pad_id
        self.bos_id = bos_id
        self.eos_id = eos_id

        self.prefix_proj = nn.Linear(embedding_dim, prefix_dim)
        self.prefix_embedding = nn.Embedding(vocab_size[0], prefix_dim, padding_idx=pad_id)
        nn.init.normal_(self.prefix_embedding.weight, mean=0.0, std=0.02)

        self.prefix_decoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'attn': RoPEMHSA(prefix_dim, prefix_dim // num_heads, prefix_dim // num_heads,
                                num_heads, dropout, rope_cache),
                'ffn': FFN(prefix_dim, ffn_dim, dropout),
                'norm1': nn.RMSNorm(prefix_dim),
                'norm2': nn.RMSNorm(prefix_dim)
            })
            for _ in range(num_layers)
        ])

        self.prefix_norm = nn.RMSNorm(prefix_dim)
        self.prefix_classifier = nn.Linear(prefix_dim, vocab_size[0])

        self.spm_proj = nn.Linear(embedding_dim, spm_dim)
        self.spm_embedding = nn.Embedding(vocab_size[1], spm_dim, padding_idx=pad_id)
        nn.init.normal_(self.spm_embedding.weight, mean=0.0, std=0.02)

        self.spm_decoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'attn': RoPEMHSA(spm_dim, spm_dim // num_heads, spm_dim // num_heads,
                                num_heads, dropout, rope_cache),
                'ffn': FFN(spm_dim, ffn_dim, dropout),
                'norm1': nn.RMSNorm(spm_dim),
                'norm2': nn.RMSNorm(spm_dim)
            })
            for _ in range(num_layers)
        ])

        self.spm_norm = nn.RMSNorm(spm_dim)
        self.spm_classifier = nn.Linear(spm_dim, vocab_size[1])

        self.suffix_proj = nn.Linear(embedding_dim, suffix_dim)
        self.suffix_embedding = nn.Embedding(vocab_size[2], suffix_dim, padding_idx=pad_id)
        nn.init.normal_(self.suffix_embedding.weight, mean=0.0, std=0.02)

        self.suffix_decoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'attn': RoPEMHSA(suffix_dim, suffix_dim // num_heads, suffix_dim // num_heads,
                                num_heads, dropout, rope_cache),
                'ffn': FFN(suffix_dim, ffn_dim, dropout),
                'norm1': nn.RMSNorm(suffix_dim),
                'norm2': nn.RMSNorm(suffix_dim)
            })
            for _ in range(num_layers)
        ])

        self.suffix_norm = nn.RMSNorm(suffix_dim)
        self.suffix_classifier = nn.Linear(suffix_dim, vocab_size[2])

        self.dropout = nn.Dropout(dropout)

    def _decode_sequence(self, input_ids, decoder_layers, final_norm, classifier):
        seq_len = input_ids.size(1)
        batch_size = input_ids.size(0)

        mask = pt.tril(pt.ones((seq_len, seq_len), device=input_ids.device, dtype=pt.bool))
        mask = mask.unsqueeze(0).expand(batch_size, -1, -1)

        x = input_ids
        for layer in decoder_layers:
            residual = x
            x_norm = layer['norm1'](x)
            x = residual + layer['attn'](x_norm, mask)

            residual = x
            x_norm = layer['norm2'](x)
            x = residual + layer['ffn'](x_norm)

        x = final_norm(x)
        logits = classifier(x)

        return logits

    def forward(self, prefix_ids, spm_ids, suffix_ids, embedding):
        embedding_proj = embedding.unsqueeze(1)

        prefix_emb_proj = self.prefix_proj(embedding_proj)
        prefix_emb = self.prefix_embedding(prefix_ids)
        prefix_input = pt.cat([prefix_emb_proj, prefix_emb], dim=1)

        prefix_logits = self._decode_sequence(
            prefix_input,
            self.prefix_decoder_layers,
            self.prefix_norm,
            self.prefix_classifier
        )[:, 1:, :]

        spm_emb_proj = self.spm_proj(embedding_proj)
        spm_emb = self.spm_embedding(spm_ids)
        spm_input = pt.cat([spm_emb_proj, spm_emb], dim=1)

        spm_logits = self._decode_sequence(
            spm_input,
            self.spm_decoder_layers,
            self.spm_norm,
            self.spm_classifier
        )[:, 1:, :]

        suffix_emb_proj = self.suffix_proj(embedding_proj)
        suffix_emb = self.suffix_embedding(suffix_ids)
        suffix_input = pt.cat([suffix_emb_proj, suffix_emb], dim=1)

        suffix_logits = self._decode_sequence(
            suffix_input,
            self.suffix_decoder_layers,
            self.suffix_norm,
            self.suffix_classifier
        )[:, 1:, :]

        return prefix_logits, spm_logits, suffix_logits

    @pt.inference_mode()
    def inference(self, embedding, max_len=64):
        self.eval()
        if isinstance(max_len, int):
            max_len = (max_len, max_len, max_len)

        device = embedding.device

        if embedding.dim() == 1:
            embedding = embedding.unsqueeze(0)
        embedding_proj = embedding.unsqueeze(1)

        prefix_ids = [self.bos_id]
        prefix_emb_proj = self.prefix_proj(embedding_proj)

        for _ in range(max_len[0]):
            curr_seq = pt.tensor([prefix_ids], device=device)
            curr_emb = self.prefix_embedding(curr_seq)
            curr_input = pt.cat([prefix_emb_proj, curr_emb], dim=1)

            logits = self._decode_sequence(
                curr_input,
                self.prefix_decoder_layers,
                self.prefix_norm,
                self.prefix_classifier
            )

            next_token = logits[0, -1].argmax(-1).item()
            prefix_ids.append(next_token)

            if next_token == self.eos_id:
                break

        spm_ids = [self.bos_id]
        spm_emb_proj = self.spm_proj(embedding_proj)

        for _ in range(max_len[1]):
            curr_seq = pt.tensor([spm_ids], device=device)
            curr_emb = self.spm_embedding(curr_seq)
            curr_input = pt.cat([spm_emb_proj, curr_emb], dim=1)

            logits = self._decode_sequence(
                curr_input,
                self.spm_decoder_layers,
                self.spm_norm,
                self.spm_classifier
            )

            next_token = logits[0, -1].argmax(-1).item()
            spm_ids.append(next_token)

            if next_token == self.eos_id:
                break

        suffix_ids = [self.bos_id]
        suffix_emb_proj = self.suffix_proj(embedding_proj)

        for _ in range(max_len[2]):
            curr_seq = pt.tensor([suffix_ids], device=device)
            curr_emb = self.suffix_embedding(curr_seq)
            curr_input = pt.cat([suffix_emb_proj, curr_emb], dim=1)

            logits = self._decode_sequence(
                curr_input,
                self.suffix_decoder_layers,
                self.suffix_norm,
                self.suffix_classifier
            )

            next_token = logits[0, -1].argmax(-1).item()
            suffix_ids.append(next_token)

            if next_token == self.eos_id:
                break

        return prefix_ids, spm_ids, suffix_ids

    def beam_search(self, embedding, beam_size=5, max_len=64, length_penalty=1.0):
        self.eval()
        if isinstance(max_len, int):
            max_len = (max_len, max_len, max_len)

        device = embedding.device

        if embedding.dim() == 1:
            embedding = embedding.unsqueeze(0)
        embedding_proj = embedding.unsqueeze(1)

        def _beam_search_component(emb_proj, embedding_fn, decoder_layers, norm_fn, classifier_fn, max_seq_len):
            sequences = [(
                [self.bos_id],
                0.0,
                None
            )]

            for step in range(max_seq_len):
                candidates = []

                for seq, score, _ in sequences:
                    if seq[-1] == self.eos_id:
                        candidates.append((seq, score, None))
                        continue

                    curr_input_ids = pt.tensor([seq], device=device)
                    curr_emb = embedding_fn(curr_input_ids)
                    curr_input = pt.cat([emb_proj, curr_emb], dim=1)

                    seq_len = curr_input.size(1)
                    attn_mask = pt.tril(pt.ones((1, seq_len, seq_len), dtype=pt.bool, device=device))

                    x = curr_input
                    for layer in decoder_layers:
                        residual = x
                        x_norm = layer['norm1'](x)
                        x = residual + layer['attn'](x_norm, attn_mask)

                        residual = x
                        x_norm = layer['norm2'](x)
                        x = residual + layer['ffn'](x_norm)

                    x = norm_fn(x)
                    logits = classifier_fn(x[0, -1])

                    log_probs = F.log_softmax(logits, dim=-1)
                    topk_log_probs, topk_indices = log_probs.topk(beam_size)

                    for i in range(beam_size):
                        token_id = topk_indices[i].item()
                        token_score = topk_log_probs[i].item()

                        new_seq = seq + [token_id]
                        new_score = score + token_score

                        if token_id == self.eos_id:
                            new_score = new_score / ((len(new_seq) - 1) ** length_penalty)

                        candidates.append((new_seq, new_score, None))

                candidates.sort(key=lambda x: x[1], reverse=True)

                sequences = candidates[:beam_size]

                if all(seq[-1] == self.eos_id for seq, _, _ in sequences):
                    break

            return sequences[0][0]

        prefix_ids = _beam_search_component(
            self.prefix_proj(embedding_proj),
            self.prefix_embedding,
            self.prefix_decoder_layers,
            self.prefix_norm,
            self.prefix_classifier,
            max_len[0]
        )

        spm_ids = _beam_search_component(
            self.spm_proj(embedding_proj),
            self.spm_embedding,
            self.spm_decoder_layers,
            self.spm_norm,
            self.spm_classifier,
            max_len[1]
        )

        suffix_ids = _beam_search_component(
            self.suffix_proj(embedding_proj),
            self.suffix_embedding,
            self.suffix_decoder_layers,
            self.suffix_norm,
            self.suffix_classifier,
            max_len[2]
        )

        return prefix_ids, spm_ids, suffix_ids


In [17]:
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super().__init__()
        self.temperature = temperature

    def forward(self, embeddings, labels=None):
        batch_size = embeddings.shape[0]

        embeddings_normalized = F.normalize(embeddings, p=2, dim=1)

        similarity_matrix = pt.matmul(
            embeddings_normalized, embeddings_normalized.transpose(0, 1)
        ) / self.temperature

        mask = pt.eye(batch_size, dtype=pt.bool, device=embeddings.device)
        similarity_matrix = similarity_matrix.masked_fill(mask, -float('inf'))

        if labels is not None:
            labels = labels.contiguous().view(-1, 1)
            mask_pos = pt.eq(labels, labels.transpose(0, 1)).float()
            mask_neg = 1.0 - mask_pos

            similarity_pos = similarity_matrix * mask_pos
            similarity_neg = similarity_matrix * mask_neg

            numerator = pt.exp(similarity_pos).sum(dim=1)
            denominator = pt.exp(similarity_matrix).sum(dim=1)

            loss = -pt.log(numerator / denominator).mean()
        else:
            logits = F.log_softmax(similarity_matrix, dim=1)
            loss = -logits.mean()

        return loss

class EnhancedAutoencoderLoss(nn.Module):
    def __init__(self, pad_id, lambda_rec=1.0, lambda_contrastive=0.2):
        super().__init__()
        self.pad_id = pad_id
        self.lambda_rec = lambda_rec
        self.lambda_contrastive = lambda_contrastive

        self.rec_criterion = nn.CrossEntropyLoss(
            ignore_index=pad_id,
            label_smoothing=0.1,
            reduction='sum'
        )

        self.contrastive_criterion = ContrastiveLoss(temperature=0.1)

    def forward(self, prefix_logits, spm_logits, suffix_logits,
                prefix_ids, spm_ids, suffix_ids, embeddings=None, word_labels=None):
        prefix_loss = self.rec_criterion(
            prefix_logits.reshape(-1, prefix_logits.shape[-1]),
            prefix_ids.reshape(-1)
        )

        spm_loss = self.rec_criterion(
            spm_logits.reshape(-1, spm_logits.shape[-1]),
            spm_ids.reshape(-1)
        )

        suffix_loss = self.rec_criterion(
            suffix_logits.reshape(-1, suffix_logits.shape[-1]),
            suffix_ids.reshape(-1)
        )

        prefix_tokens = (prefix_ids != self.pad_id).sum()
        spm_tokens = (spm_ids != self.pad_id).sum()
        suffix_tokens = (suffix_ids != self.pad_id).sum()
        total_tokens = prefix_tokens + spm_tokens + suffix_tokens

        rec_loss = (prefix_loss + spm_loss + suffix_loss) / total_tokens

        contrastive_loss = pt.tensor(0.0, device=prefix_logits.device)
        if embeddings is not None:
            contrastive_loss = self.contrastive_criterion(embeddings, word_labels)

        total_loss = self.lambda_rec * rec_loss + self.lambda_contrastive * contrastive_loss

        return total_loss, {
            'rec_loss': rec_loss.item(),
            'contrastive_loss': contrastive_loss.item() if embeddings is not None else 0.0,
            'total_loss': total_loss.item()
        }

class CurriculumSampler:
    def __init__(self, dataset, mopiece, metrics=None, initial_difficulty=0.2, epochs=50):
        self.dataset = dataset
        self.mopiece = mopiece
        self.epochs = epochs
        self.initial_difficulty = initial_difficulty
        self.current_epoch = 0

        if metrics is None:
            self.word_complexities = []
            for idx in range(len(dataset)):
                prefix_ids, spm_ids, suffix_ids = dataset[idx]
                complexity = (
                    len([i for i in prefix_ids if i != mopiece.pad_id()]) +
                    len([i for i in spm_ids if i != mopiece.pad_id()]) +
                    len([i for i in suffix_ids if i != mopiece.pad_id()])
                )
                self.word_complexities.append(complexity)

            max_complexity = max(self.word_complexities)
            self.word_complexities = [c / max_complexity for c in self.word_complexities]
        else:
            self.word_complexities = metrics

        self.indices = list(range(len(dataset)))
        self.indices.sort(key=lambda i: self.word_complexities[i])

    def update_epoch(self, epoch):
        self.current_epoch = epoch

    def get_indices(self, batch_size):
        difficulty = self.initial_difficulty + (1.0 - self.initial_difficulty) * (self.current_epoch / self.epochs)

        num_samples = int(difficulty * len(self.dataset))
        num_samples = max(batch_size, min(num_samples, len(self.dataset)))

        candidate_indices = self.indices[:num_samples]

        selected_indices = pt.randperm(len(candidate_indices))[:batch_size].tolist()
        return [candidate_indices[i] for i in selected_indices]

    def get_batch(self, batch_size):
        indices = self.get_indices(batch_size)
        batch = [self.dataset[i] for i in indices]

        prefix_batch = pt.nn.utils.rnn.pad_sequence(
            [prefix for prefix, _, _ in batch],
            batch_first=True,
            padding_value=self.mopiece.pad_id()
        )

        spm_batch = pt.nn.utils.rnn.pad_sequence(
            [spm for _, spm, _ in batch],
            batch_first=True,
            padding_value=self.mopiece.pad_id()
        )

        suffix_batch = pt.nn.utils.rnn.pad_sequence(
            [suffix for _, _, suffix in batch],
            batch_first=True,
            padding_value=self.mopiece.pad_id()
        )

        return prefix_batch, spm_batch, suffix_batch, pt.tensor(indices)

In [18]:
def train_improved_word_autoencoder(encoder, decoder, dataset, mopiece,
                              num_epochs=75, batch_size=64, lr=4e-4,
                              weight_decay=0.01, device='cuda',
                              schedule_teacher_forcing=True):
    criterion = nn.CrossEntropyLoss(
        ignore_index=mopiece.pad_id(),
        label_smoothing=0.1
    )

    dataloader = pt.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        drop_last=True, collate_fn=collate_fn
    )

    optimizer = pt.optim.AdamW([
        {"params": encoder.parameters(), "lr": lr},
        {"params": decoder.parameters(), "lr": lr}
    ], weight_decay=weight_decay)

    lr_scheduler = pt.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        steps_per_epoch=len(dataloader),
        epochs=num_epochs,
        pct_start=0.3,
        div_factor=25,
        final_div_factor=1000
    )

    history = {
        'loss': [],
        'tf_ratio': []
    }

    encoder.train()
    decoder.train()

    for epoch in (pbar := trange(num_epochs, desc='Обучение')):
        epoch_loss = 0.0

        if schedule_teacher_forcing:
            tf_ratio = max(0.5, 1.0 - 0.5 * (epoch / (num_epochs - 1)))
        else:
            tf_ratio = 1.0

        history['tf_ratio'].append(tf_ratio)

        for prefix_ids, spm_ids, suffix_ids in tqdm(dataloader, desc=f'Эпоха {epoch+1}', leave=False):
            prefix_ids = prefix_ids.to(device)
            spm_ids = spm_ids.to(device)
            suffix_ids = suffix_ids.to(device)

            optimizer.zero_grad()

            embeddings = encoder(prefix_ids, spm_ids, suffix_ids)

            embeddings = F.dropout(embeddings, p=0.1, training=True)

            prefix_logits, spm_logits, suffix_logits = decoder(
                prefix_ids[..., :-1],
                spm_ids[..., :-1],
                suffix_ids[..., :-1],
                embeddings
            )

            prefix_loss = criterion(
                prefix_logits.reshape(-1, prefix_logits.shape[-1]),
                prefix_ids[..., 1:].reshape(-1)
            )

            spm_loss = criterion(
                spm_logits.reshape(-1, spm_logits.shape[-1]),
                spm_ids[..., 1:].reshape(-1)
            )

            suffix_loss = criterion(
                suffix_logits.reshape(-1, suffix_logits.shape[-1]),
                suffix_ids[..., 1:].reshape(-1)
            )

            loss = prefix_loss + spm_loss + suffix_loss

            loss.backward()

            pt.nn.utils.clip_grad_norm_(
                list(encoder.parameters()) + list(decoder.parameters()),
                max_norm=1.0
            )

            optimizer.step()

            lr_scheduler.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        history['loss'].append(avg_loss)

        pbar.set_postfix_str(
            f'Loss: {avg_loss:.4f}, TF: {tf_ratio:.2f}, LR: {optimizer.param_groups[0]["lr"]:.6f}'
        )

        if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
            test_word_generation(encoder, decoder, dataset, mopiece, device, beam_search=True)

    return history, encoder, decoder

def test_word_generation(encoder, decoder, dataset, mopiece, device, num_samples=3, beam_search=True):
    encoder.eval()
    decoder.eval()

    indices = pt.randperm(len(dataset))[:num_samples].tolist()

    print("\n--- Тестирование ---")
    correct_count = 0

    for idx in indices:
        prefix_ids, spm_ids, suffix_ids = dataset[idx]

        prefix_ids = prefix_ids.to(device)
        spm_ids = spm_ids.to(device)
        suffix_ids = suffix_ids.to(device)

        original_word = mopiece.decode_word(
            prefix_ids.tolist()[1:-1],
            spm_ids.tolist()[1:-1],
            suffix_ids.tolist()[1:-1]
        )

        with pt.no_grad():
            embedding = encoder(
                prefix_ids.unsqueeze(0),
                spm_ids.unsqueeze(0),
                suffix_ids.unsqueeze(0)
            ).squeeze(0)

            if beam_search:
                gen_prefix_ids, gen_spm_ids, gen_suffix_ids = decoder.beam_search(
                    embedding, beam_size=5
                )
            else:
                gen_prefix_ids, gen_spm_ids, gen_suffix_ids = decoder.inference(
                    embedding
                )

            generated_word = mopiece.decode_word(
                [t for t in gen_prefix_ids[1:] if t != decoder.eos_id and t != decoder.pad_id],
                [t for t in gen_spm_ids[1:] if t != decoder.eos_id and t != decoder.pad_id],
                [t for t in gen_suffix_ids[1:] if t != decoder.eos_id and t != decoder.pad_id]
            )

            is_correct = generated_word == original_word
            if is_correct:
                correct_count += 1

            print(f"Оригинал: '{original_word}'")
            print(f"Сгенерировано: '{generated_word}' {'✓' if is_correct else '✗'}")
            print("-" * 40)

    encoder.train()
    decoder.train()


In [19]:
encoder = EnhancedWordEncoder(
    mopiece.vocab_size(),
    mopiece.pad_id(),
    embedding_dim=256,
    spm_dim=512,
    suffix_dim=256,
    prefix_dim=256,
    spm_layers=8,
    spm_heads=16,
    ffn_hidden_dim=1024,
    dropout=0.1
).to(device)

decoder = EnhancedWordDecoder(
    mopiece.vocab_size(),
    mopiece.pad_id(),
    mopiece.bos_id(),
    mopiece.eos_id(),
    embedding_dim=256,
    spm_dim=512,
    suffix_dim=256,
    prefix_dim=256,
    num_layers=8,
    num_heads=8,
    ffn_dim=1024,
    dropout=0.1
).to(device)

history, trained_encoder, trained_decoder = train_improved_word_autoencoder(
    encoder,
    decoder,
    dset,
    mopiece,
    num_epochs=100,
    batch_size=32,
    lr=3e-4,
    weight_decay=0.01,
    device=device,
    schedule_teacher_forcing=True
)

pt.save({
    'encoder_state_dict': trained_encoder.state_dict(),
    'decoder_state_dict': trained_decoder.state_dict(),
    'history': history
}, 'improved_word_autoencoder.pt')


/Users/maratgalyavov/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
Обучение:   9%|▉         | 9/100 [20:51<3:08:44, 124.45s/it, Loss: 4.0232, TF: 0.95, LR: 0.000084]


--- Тестирование генерации слов ---
Оригинал: 'fillets'
Сгенерировано: 'fillets' ✓
----------------------------------------


Обучение:  10%|█         | 10/100 [20:58<3:09:40, 126.44s/it, Loss: 4.0232, TF: 0.95, LR: 0.000084]

Оригинал: 'nahum'
Сгенерировано: 'nahum' ✓
----------------------------------------
Оригинал: 'revive'
Сгенерировано: 'revive' ✓
----------------------------------------
Точность: 100.0% (3/3)
--- Конец тестирования ---




Обучение:  19%|█▉        | 19/100 [41:12<2:44:26, 121.81s/it, Loss: 3.3682, TF: 0.90, LR: 0.000228]

...

Оригинал: 'risen'
Сгенерировано: 'rissahen' ✗
----------------------------------------
Точность: 66.7% (2/3)
--- Конец тестирования ---




Обучение:  99%|█████████▉| 99/100 [3:27:35<02:01, 121.24s/it, Loss: 3.0749, TF: 0.50, LR: 0.000000]


--- Тестирование генерации слов ---
Оригинал: 'ammah'
Сгенерировано: 'ammah' ✓
----------------------------------------


Обучение: 100%|██████████| 100/100 [3:27:36<00:00, 124.56s/it, Loss: 3.0749, TF: 0.50, LR: 0.000000]

Оригинал: 'vehemently'
Сгенерировано: 'vehement' ✗
----------------------------------------
Оригинал: 'throughout'
Сгенерировано: 'throughoutp' ✗
----------------------------------------
Точность: 33.3% (1/3)
--- Конец тестирования ---

